In [1]:
%load_ext autoreload
%autoreload 2
%load_ext jupyter_black
%matplotlib inline
import os

os.environ["PATH_TO_SEVIR"] = "/mnt/data/sevir"  # you can set the default path via an environment variable
import sevir as sevir
from sevir.constants import VIS, IR_069, VIL, IR_107, LGHT

# TensorGenerator

The tensor generator will automatically create the catalog if one was not provided.
If you want to do some more filtering of the catalog data prior to creating the tensor
generator, it accepts a catalog as as the first argument.

This generator will split the catalog by the provided image types, a patch size argument,
can also be passed to interpolate the LxW of the image. So that the multiple images can 
fit into a single tensor.



In [2]:
# the generator will reshape the  height and width to the largest of the batch by default
with sevir.TensorGenerator(inputs=(IR_069, IR_107, VIS), features=(VIL,)) as tg:
    (x, y), df = tg.select("R19082620197425", metadata=True)
    print("torch.Size([C,   L,   W,  T])", x.shape, y.shape, sep="\n")

df

INFO:root:Loading 101 files with 3 image types.
100%|██████████| 101/101 [00:02<00:00, 35.99it/s]
INFO:root:Loading 10 files with 1 image types.
100%|██████████| 10/10 [00:00<00:00, 38.04it/s]
INFO:root:🏪 Closing Store 🏪


torch.Size([C,   L,   W,  T])
torch.Size([3, 768, 768, 49])
torch.Size([1, 768, 768, 49])


id,file_name,file_index,img_type,time_utc,minute_offsets,episode_id,event_id,event_type,llcrnrlat,llcrnrlon,urcrnrlat,urcrnrlon,proj,size_x,size_y,height_m,width_m,data_min,data_max,pct_missing,file_ref
str,str,i64,str,datetime[μs],str,f64,f64,str,f64,f64,f64,f64,str,i64,i64,f64,f64,f64,f64,f64,i32
"""R1908262019742…","""/mnt/data/sevi…",564,"""ir069""",2019-08-26 20:19:00,"""-120:-115:-110…",null,null,null,28.845626,-87.997107,31.833639,-83.559175,"""+proj=laea +la…",192,192,384000.0,384000.0,-76.769867,-22.281887,0.0,81
"""R1908262019742…","""/mnt/data/sevi…",564,"""ir107""",2019-08-26 20:19:00,"""-120:-115:-110…",null,null,null,28.845626,-87.997107,31.833639,-83.559175,"""+proj=laea +la…",192,192,384000.0,384000.0,-76.141235,23.99688,0.0,31
"""R1908262019742…","""/mnt/data/sevi…",150,"""vis""",2019-08-26 20:19:00,"""-120:-115:-110…",null,null,null,28.845626,-87.997107,31.833639,-83.559175,"""+proj=laea +la…",768,768,384000.0,384000.0,0.012209,1.214073,0.00181,68
"""R1908262019742…","""/mnt/data/sevi…",766,"""vil""",2019-08-26 20:19:35,"""-120:-115:-110…",null,null,null,28.845626,-87.997107,31.833639,-83.559175,"""+proj=laea +la…",384,384,384000.0,384000.0,0.0,247.0,0.0,0


In [3]:
with sevir.TensorLoader(
    sevir.TensorGenerator(inputs=(IR_069, IR_107, VIS), features=(VIL,), patch_size=48),
    batch_size=5,
) as loader:
    x, y = next(iter(loader))
    print("torch.Size([B, C,  X,  Y,  T])", x.shape, y.shape, sep="\n")

INFO:root:Loading 101 files with 3 image types.
100%|██████████| 101/101 [00:01<00:00, 86.27it/s]
INFO:root:Loading 10 files with 1 image types.
100%|██████████| 10/10 [00:00<00:00, 72.08it/s]
INFO:root:⛈️ Beginning Tensor Generation ⛈️
  0%|          | 4/12741 [00:11<9:44:44,  2.75s/it]
INFO:root:🏪 Closing Store 🏪


torch.Size([B, C,  X,  Y,  T])
torch.Size([5, 3, 48, 48, 49])
torch.Size([5, 1, 48, 48, 49])
